In [1]:
import cv2

In [2]:
import numpy as np
import matplotlib
matplotlib.use("TkAgg") # have to use this for tkinter to  work below
from matplotlib import pyplot as plt
%matplotlib tk

# scikit image
import skimage
from skimage import io

import os

In [3]:
%qtconsole

In [4]:
# list files in director

In [5]:
os.chdir("/Users/callinswitzer/Dropbox/KalmiaProject/KalmiaFruits_28Sept2016/")

In [6]:
# list files in directory
mypath = os.getcwd()
onlyfiles = [f for f in os.listdir(mypath) if os.path.isfile(os.path.join(mypath, f))]

In [7]:
photoList = [ x for x in onlyfiles if "DSC" not in x and not x.startswith('.') and x.endswith('.jpg')]

In [192]:
# read in image as greyscale
#kk = 10
kk = kk + 1
fpth = photoList[kk]
img = cv2.imread(fpth,0)

In [193]:
plt.clf()
plt.xlim(6000, 0)
plt.ylim(0, 4000)
io.imshow(img) # so fast, compared to sklearn

In [194]:
# Otsu's thresholding
ret2,th2 = cv2.threshold(img,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

# Otsu's thresholding after Gaussian filtering
blur = cv2.GaussianBlur(img,(5,5),0)
ret3,th3 = cv2.threshold(blur,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

In [195]:
# now get rid of the leafy bits
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
erosion = cv2.erode(th3,kernel,iterations = 10)
io.imshow(erosion)

In [196]:
dilation = cv2.dilate(erosion, kernel, iterations = 10)
plt.imshow(dilation, cmap = 'gray')

In [197]:
# close small holes

# find bad contours
def is_contour_bad(c):
    # approximate the contour
    area1 = cv2.contourArea(c, False) 

    # the contour is 'bad' if it is really small
    return area1 > 1000

In [198]:
image = dilation.copy()
io.imshow(image)

(cnts, _) = cv2.findContours(image.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
mask = np.ones(image.shape[:2], dtype="uint8") * 0 # create a blank black mask



In [199]:
# loop over the contours
for c in cnts:
    # if the contour is bad, draw it on the mask (in white)
    if is_contour_bad(c):
        cv2.drawContours(mask, [c], -1, 255, -1)
        
# see if any bad contours were found
filHoles = 'no holes were filled--' + str((mask == image).all()) # will be false if a small hole was filled

In [200]:
# remove the contours from the image and show the resulting images
image = cv2.bitwise_or(image, mask)
io.imshow( mask) # mask

In [201]:
io.imshow( image) # after

In [202]:
# find contours
#im2, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
cts, im2 = cv2.findContours(image.copy(),  cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

In [203]:
# draw all contours (light)
plt.clf()
plt.xlim(6000, 0)
plt.ylim(0, 4000)
cv2.drawContours(img, cts, -1, (140,20,0), -1) # note, this modifies the original image, "img"
io.imshow(img)

In [204]:
# same as below, but puts it in a list
# for ii in range(len(cts)):
#     print cv2.arcLength(cts[ii], True) 

perimeters = [cv2.arcLength(cts[ii], True) for ii in range(len(cts))]


In [205]:
# calculate areas
areas = [cv2.contourArea(cts[ii], False) for ii in range(len(cts))]


# calculate circularity
circularity = (perimeters .^ 2) ./ (4 * pi * area)
If this is closer to 1, then the object is more circle-like

In [206]:
circularity = [(x **2) / (4*3.14159*y) for x,y in zip(perimeters, areas)]


In [207]:
arrs = np.array([i < 1.5 for i in circularity]) # get only roughly circular objects
# circles will be close to 0


In [208]:
ctd = np.array(cts)
filt_contours = ctd[arrs]

In [209]:
# draw filtered contours (white)
plt.clf()
plt.xlim(6000, 0)
plt.ylim(0, 4000)
cv2.drawContours(img, filt_contours, -1, (255,255,0), -1) # note, this modifies the original image, "img"
io.imshow(img)

In [210]:
# calculate diameters in pixels
# area = pi * r ^2
# r = sqrt(area / pi)

radii = np.array([np.sqrt(aa) for aa in areas])

In [211]:
# show circle with largest diameter (black)
bgCirc = np.array(radii == radii.max())

# draw filtered contours (black)
cv2.drawContours(img, ctd[bgCirc], -1, (0,255,0), -1) # note, this modifies the original image, "img"
io.imshow(img)

In [212]:
radii[bgCirc]

array([ 622.89485469])

In [213]:
# compute the center of the contour
Moms = np.array([cv2.moments(cc) for cc in cts])
# get xy coordinates
cs = np.array([[int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"])] for M in Moms])


In [214]:
xx = np.array([cs[ii, 0] for ii in range(len(cs))])
yy = np.array([cs[ii, 1] for ii in range(len(cs))])

In [215]:
io.imshow(img)

In [216]:
plt.plot(xx, yy, 'bp')
plt.xlim(6000, 0)
plt.ylim(0, 4000)
plt.show()

In [217]:
print len(xx) # number of fruits (including big circle)
print filHoles

13
no holes were filled--True
